In [17]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 인간의 대화를 이해하고 돕습니다."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [22]:
invoke_chain("내 이름은 김희준입니다.")

content='안녕하세요, 김희준님! 만나서 반가워요. 무엇을 도와드릴까요?'


In [23]:
invoke_chain("나의 이름은 뭔가요?")

content='당신의 이름은 김희준입니다.'




> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 인간의 대화를 이해하고 돕습니다.
System: The human introduces themselves as Kim Heejoon. The AI responds in Korean, greeting Kim Heejoon and asking how it can help. Kim Heejoon reveals they are a programmer and the AI expresses admiration for their profession. The AI asks about the programming languages and technologies Kim Heejoon uses and what kind of help they need. Kim Heejoon shares they are 45 years old and the AI acknowledges their experience in the field. The AI asks about the specific area Kim Heejoon works in and what assistance they require, offering to help with any gaps in knowledge or questions they may have.
Human: 내 이름이 뭐지?

> Finished chain.


'당신의 이름은 김희준이에요. 어떻게 도와드릴까요?'

In [20]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='안녕하세요'),
  AIMessage(content='안녕하세요! 무엇을 도와드릴까요?')]}